> Texto fornecido sob a Creative Commons Attribution license, CC-BY. Todo o código está disponível sob a FSF-approved BSD-3 license.<br>
> (c) Original por Lorena A. Barba, Gilbert F. Forsyth em 2017, traduzido por Felipe N. Schuch em 2020.<br>
> [@LorenaABarba](https://twitter.com/LorenaABarba) - [@fschuch](https://twitter.com/fschuch)

12 passos para Navier-Stokes
======
***

Essa tarefa é um complemento para as aulas do primeiro módulo interativo online [CFD com Python](https://github.com/fschuch/CFDPython-BR), por Prof. Lorena A. Barba, denominado **12 Passos para Navier-Stokes**. Esse notebook foi escrito pelo estudante de graduação Gilbert Forsyth.

Operações com arranjos em NumPy
----------------

Para aplicações computacionais mais intensivas, o uso das funções embutidas em NumPy podem fornecer um aumento na velocidade de execução de muitas vezes. Como um exemplo simples, considere a seguinte equação:

$$
u^{n+1}_i = u^n_i-u^n_{i-1}
$$

Agora, para um dado vetor $u^n = [0, 1, 2, 3, 4, 5]\ \ $, nós podemos calcular o valor de $u^{n+1}$ ao iterar sobre os valores de $u^{n+1}$ com um laço `for`.

In [1]:
import numpy

In [2]:
u = numpy.array((0, 1, 2, 3, 4, 5))

for i in range(1, len(u)):
    print(u[i] - u[i-1])

1
1
1
1
1


Esse é o resultado esperado, e o tempo de execução foi praticamente instantâneo. Se efetuamos o mesmo procedimento como uma operação de arranjos, em vez de calcularmos $u^n_i-u^n_{i-1}$ separadamente por 5 vezes, podemos fatiar o arranjo $u$ e calcular cada operação com uma linha de comando:

In [3]:
u[1:] - u[0:-1]

array([1, 1, 1, 1, 1])

O que esse comando diz é para subtrair o 0.º, 1.º, 2.º, 3.º, 4.º e 5.º elementos de $u$ do 1.º, 2.º, 3.º, 4.º, 5.º e 6.º elementos de $u$.

### Aumento de Velocidade

Para o arranjo de 6 elementos, o benefício da operação de arranjos é bastante pequena. Não haverá diferença significativa no tempo de execução porque existem apenas algumas poucas operações ocorrento. Mas se revisitarmos a equação de convecção linear 2D, podemos ver um ganho de velocidade substancial.

In [4]:
nx = 81
ny = 81
nt = 100
c = 1
dx = 2 / (nx - 1)
dy = 2 / (ny - 1)
sigma = .2
dt = sigma * dx

x = numpy.linspace(0, 2, nx)
y = numpy.linspace(0, 2, ny)

u = numpy.ones((nx, ny)) ##Cria um vetor ny x nx com 1
un = numpy.ones((nx, ny)) 

###Assinala a condição Inicial

u[int(.5 / dx):int(1 / dx + 1), int(.5 / dy): int(1 / dy + 1)] = 2

Com nossa condição inicial definida, vamos primeiro executar a forma original de fois laços `for` aninhados, fazendo uso da função "mágica" do Notebook `%%timeit`, a qual vai nos ajudar a mensurar a performace do nosso código.

**Nota:** A função mágica `%%timeit` vai executar o código diversas vezes e nos fornecer como resultado o tempo médio de execução. Se tivermos uma figura sendo produzida dentro da célula onde executamos `%%timeit`, ela será executada repetitivamente, o que pode causar uma leve confusão.

O tempo de execução abaixo vai variar de máquina para máquina. Não espere que seus tempos correspondam e esses, mas você _deve_ verificar a mesma tendência geral de diminuição no tempo de execução conforme mudamos para operações de arranjos.

In [5]:
%%timeit
u = numpy.ones((nx, ny))
u[int(.5 / dx):int(1 / dx + 1), int(.5 / dy): int(1 / dy + 1)] = 2

for n in range(nt + 1): ##Laço sobre o número de passos de tempo
    un = u.copy()
    row, col = u.shape
    for i in range(1, row):
        for j in range(1, col):
            u[i, j] = (un[i, j] - (c * dt / dx * 
                                  (un[i, j] - un[i - 1, j])) - 
                                  (c * dt / dy * 
                                  (un[i, j] - un[i, j - 1])))
            u[0, :] = 1
            u[-1, :] = 1
            u[:, 0] = 1
            u[:, -1] = 1

2.42 s ± 24.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Com o código "bruto" acima, observe a média de execução atingida. Tenha em mente que com esses três laços aninhados, que tudo declarado dentro do laço **j** está sendo executado mais de 650.000 vezes. Vamos comparar isso com o desempenho do mesmo código implementado com operações de arranjos:

In [6]:
%%timeit
u = numpy.ones((ny, nx))
u[int(.5 / dx):int(1 / dx + 1), int(.5 / dy): int(1 / dy + 1)] = 2

for n in range(nt + 1): ##Laço sobre o número de passos 
    un = u.copy()
    u[1:, 1:] = (un[1:, 1:] - (c * dt / dx * (un[1:, 1:] - un[0:-1, 1])) -
                              (c * dt / dy * (un[1:, 1:] - un[1, 0:-1])))
    u[0, :] = 1
    u[-1, :] = 1
    u[:, 0] = 1
    u[:, -1] = 1

6.2 ms ± 41.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Como você pode ver, o tempo de execução caiu substancialmente. Pode não ter parecido muito tanto nesse exemplo, mas tenha em mente que o ganho de velocidade vai crescer exponencialmente com o tamanho e complexidade do problema que está sendo resolvido.

In [7]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

> A célula acima executa o estilo para esse notebook. Nós modificamos o estilo encontrado no GitHub de [CamDavidsonPilon](https://github.com/CamDavidsonPilon), [@Cmrn_DP](https://twitter.com/cmrn_dp).